#**Implementation of a simple MIA**

Membership Inference Attack is one of the most popular methods to reveal (to some extent) whether a model has been trained on a particular training example.




*   Please goto Runtime -> Change runtime type -> Hardware accelerator -> T4 GPU.
*   Install the following packages

In [2]:
!pip install torch==2.1.0
!pip install torchtext==0.14.0
!pip install datasets

  Using cached torchtext-0.14.0-cp310-cp310-manylinux1_x86_64.whl.metadata (6.9 kB)
  Using cached torch-1.13.0-cp310-cp310-manylinux1_x86_64.whl.metadata (23 kB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
Using cached torchtext-0.14.0-cp310-cp310-manylinux1_x86_64.whl (2.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 782.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#**Importing the necassary packages**

In [1]:
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np


#Loading the `imdb` dataset:

> This dataset contains the movie reviews and the sentiment of the review.
I chose this dataset as the task is to perform the membership inference attack in NLP. Also it can be easily loaded from `datasets`.




In [2]:
# Here I'm loading the IMDB dataset
dataset = load_dataset("imdb")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [3]:
# To tokenize the reviews I have used the torch tokenizer, get_tokenizer
tokenizer = get_tokenizer("basic_english")

# defining the preprocessing function to tokenize the data
def preprocess_data(dataset):
    texts = [tokenizer(text) for text in dataset['text']]
    labels = dataset['label']
    return list(zip(texts, labels))

# preprocessing the training and the test data
train_data = preprocess_data(dataset['train'])
test_data = preprocess_data(dataset['test'])


In [4]:
# To create the vocabulary, I used inbuilt torchtext build_vocab_from_iterator function
vocab = build_vocab_from_iterator([text for text, label in train_data], specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])


In [5]:
# defining fundtion to convert the tokens into tensor to get interger ID using vocabulary
def text_to_tensor(text):
    return torch.tensor([vocab[token] for token in text], dtype=torch.long)


In [6]:
# The reviews can be of different lengths and the collate_fn function is defined to pad the input sequence into same length
def collate_fn(batch):
    texts, labels = zip(*batch)

    # Padding the sequences to the same length
    # batch_first=True means we make the tensor shape: (batch_size, seq_len))
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=0)

    # Converting the labels into tensor
    padded_labels = torch.tensor(labels, dtype=torch.float)

    return padded_texts, padded_labels

# Converting training and test data to tensors
train_tensor_data = [(text_to_tensor(text), label) for text, label in train_data]
test_tensor_data = [(text_to_tensor(text), label) for text, label in test_data]

# Creating DataLoader with custom collate_fn to process the data efficiently
# We want to pass samples in “minibatches”, reshuffle the data at every epoch to reduce model overfitting
train_loader = DataLoader(train_tensor_data, batch_size=64, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_tensor_data, batch_size=64, shuffle=True, collate_fn=collate_fn)


The following is the target model arcitecture which is a multi-layer percetron model for text classification. It contains three layers

*   Embedding layer
*   Two fully connected layers



In [7]:
class MLPTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(MLPTextClassifier, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # Defining a first fully connected layer
        self.fc1 = nn.Linear(embed_dim, hidden_dim)

        #Defining the activation function ReLu
        self.relu = nn.ReLU()

        # Defininf the final output layer
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Applying the embedding layer
        x = self.embedding(x)

        # Flattened the sequence of tokens by taking mean to make it fixd-size vector
        x = x.mean(dim=1)

        # Passing the sequence to the two fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x



In [8]:
# selecting the available device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

I have defined below the model hyperparameters and instantiated the target model

In [9]:
# Defining the model hyperparameters
vocab_size = len(vocab)  # Size of the vocabulary
embed_dim = 100  # Dimension of the word embeddings
hidden_dim = 128  # Size of hidden layer
output_dim = 1  # Binary classification (positive/negative)

# Instantiating the target model
target_model = MLPTextClassifier(vocab_size, embed_dim, hidden_dim, output_dim).to(device)


In [10]:
# Defining the optimizer and loss function
optimizer = optim.Adam(target_model.parameters())
criterion = nn.BCEWithLogitsLoss() # As binary classification

# Training loop for the target model
def train_model(model, train_loader, optimizer, criterion):
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for texts, labels in train_loader:
        texts = texts.to(device)  # for faster execution sending to device (GPU or CPU)
        labels = labels.to(device)  # Similarly sedning the labels to device

        optimizer.zero_grad()

        # Forward pass
        outputs = model(texts)
        loss = criterion(outputs.squeeze(1), labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)

# Training the target model for 5 epochs
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_model(target_model, train_loader, optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs}, The training loss of the target model is: {train_loss:.4f}")


Epoch 1/5, The training loss of the target model is: 0.6858
Epoch 2/5, The training loss of the target model is: 0.5273
Epoch 3/5, The training loss of the target model is: 0.3667
Epoch 4/5, The training loss of the target model is: 0.2894
Epoch 5/5, The training loss of the target model is: 0.2400


I have defined the shadow model below which mimics the architecture and the data used in the target model. The attackers uses the shadow model to learn pattens in how models behave on in-sample vs out of the sample data to perform membership inference attack.

In [11]:
# Defining the shadow model which mimics the behaviour of the target model
shadow_model = MLPTextClassifier(vocab_size, embed_dim, hidden_dim, output_dim).to(device)

# Defining the same optimizer and loss function for the shadow model so that it behaves similarly like the target model to attack
shadow_optimizer = optim.Adam(shadow_model.parameters())
shadow_criterion = nn.BCEWithLogitsLoss()

# Training loop for the shadow model
# using the same data to train the model
def train_shadow_model(model, train_loader, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for texts, labels in train_loader:
        texts = texts.to(device)  # for faster execution sending to device (GPU or CPU)
        labels = labels.to(device)  # Similarly sedning the labels to device

        optimizer.zero_grad()

        # Forward pass
        outputs = model(texts)
        loss = criterion(outputs.squeeze(1), labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)

# Trainning the shadow model for 5 epochs
num_epochs = 5
for epoch in range(num_epochs):
    shadow_loss = train_shadow_model(shadow_model, train_loader, shadow_optimizer, shadow_criterion)
    print(f"Epoch {epoch+1}/{num_epochs}, The trainning loss of the Shadow Model is: {shadow_loss:.4f}")


Epoch 1/5, The trainning loss of the Shadow Model is: 0.6870
Epoch 2/5, The trainning loss of the Shadow Model is: 0.5321
Epoch 3/5, The trainning loss of the Shadow Model is: 0.3637
Epoch 4/5, The trainning loss of the Shadow Model is: 0.2865
Epoch 5/5, The trainning loss of the Shadow Model is: 0.2441


Now, the attacks tried to infer whether the given data point was in training data or not.


*   The in_sample_data and the out_sample_data are used for predictions.
*   These predictios are converted into probabilities using the sigmoid function to provide confidence score for every perdiction.



In [26]:
# Using the training data as in-sample data as these are used for training i.e. the members
i_s_data = train_tensor_data
#in_sample_data

In [27]:
# Using the testing data as out-of-sample data because these were not seen by the model during tranining hence it is a non-member part of the tarining data
o_s_data = test_tensor_data

In [28]:
# Preparing the DataLoader to process the data efficintly
i_s_loader = DataLoader(i_s_data, batch_size=64, collate_fn=collate_fn, shuffle=True)
o_s_loader = DataLoader(o_s_data, batch_size=64, collate_fn=collate_fn, shuffle=False)

In [29]:
i_s_loader

In [30]:
# Here the model is being tested and the predictions are appended to get the logit value of the predictions
def get_logits(model, data_loader):
    model.eval()
    all_logits = []
    with torch.no_grad():
        for batch in data_loader:
            texts, _ = batch
            texts = texts.to(device)
            outputs = model(texts)
            all_logits.append(outputs.cpu().numpy())
    return np.concatenate(all_logits)

# Processing to get the logit values of the in_sample and out_sample data
i_s_logits = get_logits(shadow_model, i_s_loader)
o_s_logits = get_logits(shadow_model, o_s_loader)

In [35]:
i_s_logits.shape

(25000, 1)

Now I got all the logits for in_sample and out_saple data. The following function is defined to get top 90% confident data.  

In [31]:
import numpy as np

# The following function is defined to get the top logits with 90% confidence
def top_confident_samples(logits, labels, top_percent):

    confidences = np.max(np.abs(logits), axis=1)  # Confidence = max absolute logit value
    num_top_samples = int(len(confidences) * top_percent)

    # Sorting the indices to get the top logits
    top_indices = np.argsort(confidences)[-num_top_samples:]
    return logits[top_indices], labels[top_indices]

# in-sample data and out-sample sample data is created and annotated with 1 and 0
# in-sample is annotated with 1 as it belongs to training data
# out-sample is annotated with 0 as it does not belong to the training data
topILogits, topILabels = top_confident_samples(i_s_logits, np.ones(len(i_s_logits)), top_percent=0.9)
topOLogits, topOLabels = top_confident_samples(o_s_logits, np.zeros(len(o_s_logits)), top_percent=0.9)

# Combining them all to get the filtered in-sample and out-sample logits for training the attack model
X_attack_top = np.concatenate([topILogits, topOLogits])
y_attack_top = np.concatenate([topILabels, topOLabels])

In [32]:
from sklearn.model_selection import train_test_split

# Splitting the shuffled data for fair allocation of the in-sample and out_sample in train and test data for attack model
X_train, X_test, y_train, y_test = train_test_split(X_attack_top, y_attack_top, shuffle= True, test_size=0.3, random_state=42)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# XGBoost classifier is initialized as an attack model
mia_classifier = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
mia_classifier.fit(X_train, y_train)

# Predictions and probabilities are calcuated
y_pred = mia_classifier.predict(X_test)
y_proba = mia_classifier.predict_proba(X_test)[:, 1]

# Evaluting the performance of the attack model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"MIA Classifier Accuracy: {accuracy:.4f}")
print(f"MIA Classifier ROC-AUC: {roc_auc:.4f}")

MIA Classifier Accuracy: 0.5613
MIA Classifier ROC-AUC: 0.5772


Reference: https://drlee.io/ai-security-membership-inference-attacks-and-mitigating-them-with-differential-privacy-with-code-78bf3f7af5d8